In [1]:
import pyspark
from pyspark.sql import SparkSession
#from Funciones import *
from pyspark.sql.window import Window as W
from elasticsearch import Elasticsearch
import nltk
from tqdm import tqdm
import pandas as pd
import textdistance
import time
from pyspark.sql import functions as F
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import FloatType, StructField, StringType, StructType

In [2]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/30 12:01:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/30 12:01:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/30 12:01:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
Nombre_Input_entidades = spark.read.csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", sep=";", header=True)
Nombre_Input_relaciones = spark.read.csv("DATA_RELACION_ENTIDADES_MAESTRO.csv", sep=";", header=True)
Nombre_Input_investigadores = spark.read.csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv", sep=";", header=True)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/arturo/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
col = list(Nombre_Input_investigadores.select("Id","Entidad").toPandas().columns)

In [ ]:
columnsSourcesNames1 = [[]]*len(list(Nombre_Input_investigadores.select("Id","Entidad").toPandas().columns))

In [ ]:
def format_results(results,df1columns,df2columns):
    names_source = []
    names_target = []
    columnsSourcesNames1 = [[]]*len(df1columns)
    columnsTargetNames1 = [[]]*len(df2columns)
    
    i=0
    for i in results:
        
        source_name = results[i]['source_name']
        columnsSourcesNamesResults = [None]*len(df1columns)
        
        for l in range(len(df1columns)):
            columnsSourcesNamesResults[l] = results[i][df1columns[l]]

        for element in results[i]['matching_names_by_elasticsearch']:
            target_name = element[1]
            names_source.append(source_name)
            names_target.append(target_name)
            

            for k in range(len(df1columns)):
                columnsSourcesNames1[k] = columnsSourcesNames1[k] + [columnsSourcesNamesResults[k]] 

            columnsTargetNamesResults = [None]*len(df2columns)
            
            for m in range(len(df2columns)):
                columnsTargetNamesResults[m] = element[m+2]
                
            for j in range(len(df2columns)):
                columnsTargetNames1[j] = columnsTargetNames1[j] + [columnsTargetNamesResults[j]] 
                
    dicCol = {}
    for n in range(len(df1columns)):
        dicCol[df1columns[n]] = columnsSourcesNames1[n]
    
    data = {'source_name':names_source, 'target_name':names_target}
    
    data.update(dicCol)
    
    dicCol = {}
    for n in range(len(df2columns)):
        dicCol[df2columns[n]] = columnsTargetNames1[n]
    
    data.update(dicCol)
    
    result = pd.DataFrame(data)
            
    return result


In [ ]:
def Distance_ratcliff_obershelp(column1, column2):
    return textdistance.ratcliff_obershelp(column1, column2)

udf_Distance_ratcliff_obershelp = F.udf(Distance_ratcliff_obershelp, FloatType())

In [ ]:
def index_es(df,target_name_column,index_name):
    '''
    inputs:
        df : pd.DataFrame object containing the two columns source names and target names
        target_names : type: str, represents the the target names column
        index_name : type: str, represents the index of elastic search object
    output:
        return elasticsearch object indexed, and initialized with target names
    
    '''

    target_names = [name.lower() for name in list(df[target_name_column])]
    es = Elasticsearch()
    
    for i in list(es.indices.get_alias("*").items()):
        es.indices.delete(index=list(i)[0])
    
    es.indices.delete(index='test-index', ignore=[400, 404])
    
    
    i=0
    for name in tqdm(target_names):
        doc = {'entity':name}
        es.index(index=index_name, id=i, document=doc)
        i+=1    
    return es



In [ ]:
def get_matching_entities_by_elasticSearch(df1,
                                           source_name_column,
                                           df1columns,
                                           df2,
                                           target_name_column,
                                           df2columns,
                                           index_name,
                                           n):
    '''
    inputs:
        df : pd.DataFrame object containing the two columns source names and target names
        source_names : type: str, represents the the source names column
        index_name : type: str, represents the index of elastic search object
        size: type: int , number of targets to extract by elasticseasource_pd_altas_nuevas['source_municipality'] = source_pd_altas_nuevas['source_municipality'].astype('Int64')rch, default is 20,
                max for this paremeter is 10 000 
    output:
        return dict object results
    '''
    
    df1 = df1.toPandas().head(1000)
    df2 = df2.toPandas().head(1000)
    
    es = index_es(df2,target_name_column,index_name)

    source_name = list(df1[source_name_column])
    
    columnsSourcesNames = [[]]*len(df1columns)
    for k in range(len(df1columns)):
        columnsSourcesNames[k] = list(df1[df1columns[k]])
        
    columnstargetNames = [[]]*len(df2columns)
    for k in range(len(df2columns)):
        columnstargetNames[k] = list(df2[df2columns[k]])
    
    source_idx=0
    results = {}

    for entity_i in tqdm(source_name):

        matching_entities = []
        query = {"from": 0,
                 "size": n,
                 "query": {"match": {"entity":{'query':entity_i,'fuzziness':'AUTO'}}},
                 "sort":[{'_score':{'order':'desc'}}]}
        resp = es.search(index=index_name, body=query)

        for element in resp['hits']['hits']:

            target_idx = int(element['_id'])
            entity_j = element['_source']['entity']
            
            firstMatch = [target_idx,entity_j]
            
            secondMatch = []
            
            for l in range(len(df2columns)):
                secondMatch = secondMatch + [columnstargetNames[l][target_idx]]
            
            match = tuple(firstMatch + secondMatch)
            matching_entities.append(match)
        
        dicCol = {}
        
        for l in range(len(df1columns)):
            dicCol[df1columns[l]] = columnsSourcesNames[l][source_idx]
        

        results[source_idx] = {'source_name':entity_i, 'matching_names_by_elasticsearch':matching_entities}
        results[source_idx].update(dicCol)
        source_idx +=1
    results = format_results(results, df1columns,df2columns)    
    return results


In [18]:
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Input_investigadores, 
                                        "Entidad_Norm", 
                                        ["Id","CIF"],
                                        Nombre_Input_entidades, 
                                        "Entidad_Norm",
                                        ["Provincia_Entidad", "Entidad_Norm","List_Entidad_Norm"],
                                        'index_name',
                                        1)

/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/ipykernel_63852/3719534698.py:15: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  for i in list(es.indices.get_alias("*").items()):
/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
  0%|                                                  | 0/1000 [00:00<?, ?it/s]/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/ipykernel_63852/2274864580.py:45: DeprecationWarning: The 'body' parameter is deprecated for t

In [19]:
def get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,results):
    
    print(0)
    results = spark.createDataFrame(results)
    print(1)    
    stopwords_es = nltk.corpus.stopwords.words('spanish')
    print(2) 
    results = results.withColumn("aux", F.split("source_name", "\\s+"))
    print(3) 
    remover = StopWordsRemover(stopWords=stopwords_es, inputCol="aux", outputCol="source_names_stopwords")
    print(4) 
    results = remover.transform(results).withColumn("source_names_stopwords", F.array_join("source_names_stopwords", " "))
    print(5) 
    results = results.withColumn("aux", F.split("target_name", "\\s+"))
    print(6) 
    remover = StopWordsRemover(stopWords=stopwords_es, inputCol="aux", outputCol="target_names_stopwords")
    print(7) 
    results = remover.transform(results).withColumn("target_names_stopwords", F.array_join("target_names_stopwords", " "))
    print(8) 
    return results.withColumn('final_score', udf_Distance_ratcliff_obershelp(F.col('source_names_stopwords'), F.col('target_names_stopwords'))).drop('aux').withColumn('final_score', F.when(F.col('source_names_stopwords') != (F.col('target_names_stopwords')), F.col('final_score')).otherwise(F.lit(1)))

In [20]:
d = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l)

0
1
2
3
22/07/30 11:23:54 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
4
5
6
22/07/30 11:23:54 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
7
8


In [22]:
d.toPandas().head(15)

,source_name,target_name,Id,CIF,Provincia_Entidad,Entidad_Norm,List_Entidad_Norm,source_names_stopwords,target_names_stopwords,final_score
0,agencia estatal consejo superior de investigac...,consejo de la juventud de extremadura,237,Q2818002D,6,consejo de la juventud de extremadura,['consejo de la juventud de extremadura'],agencia estatal consejo superior investigacion...,consejo juventud extremadura,0.301075
1,agencia estatal consejo superior de investigac...,consejo de la juventud de extremadura,237,Q2818002D,6,consejo de la juventud de extremadura,['consejo de la juventud de extremadura'],agencia estatal consejo superior investigacion...,consejo juventud extremadura,0.301075
2,fundacion parc tauli,instituto de investigacion e innovacion parc t...,1886,G60331238,8,instituto de investigacion e innovacion parc t...,['instituto de investigacion e innovacion parc...,fundacion parc tauli,instituto investigacion innovacion parc tauli,0.553846
3,fundacion prodintec,fundacion barcelona mobile world capital found...,642,G33914557,8,fundacion barcelona mobile world capital found...,['fundacion barcelona mobile world capital fou...,fundacion prodintec,fundacion barcelona mobile world capital found...,0.428571
4,universidad de navarra,gobierno de navarra,900,R3168001J,31,gobierno de navarra,['gobierno de navarra'],universidad navarra,gobierno navarra,0.628571
5,fundacion centro tecnologico de componentes,fundacion centac centro nacional de tecnologia...,656,G39492947,28,fundacion centac centro nacional de tecnologia...,['fundacion centac centro nacional de tecnolog...,fundacion centro tecnologico componentes,fundacion centac centro nacional tecnologias a...,0.612245
6,universidad autonoma de madrid,universidad nacional autonoma de mexico,836,Q2818013A,77,universidad nacional autonoma de mexico,['universidad nacional autonoma de mexico'],universidad autonoma madrid,universidad nacional autonoma mexico,0.730159
7,universidad miguel hernandez de elche,fundacion miguel servet,860,Q5350015C,31,fundacion miguel servet,"['fundacion miguel servet navarrabiomed', 'fun...",universidad miguel hernandez elche,fundacion miguel servet,0.526316
8,universidad de zaragoza,instituto agronomico mediterraneo de zaragoza,880,Q5018001G,50,instituto agronomico mediterraneo de zaragoza,['instituto agronomico mediterraneo de zaragoza'],universidad zaragoza,instituto agronomico mediterraneo zaragoza,0.483871
9,universidad de alcala,universidad de alcala,832,Q2818018J,28,universidad de alcala,"['universidad de alcala uah', 'universidad de ...",universidad alcala,universidad alcala,1.000000


In [381]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arturo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [364]:
results = spark.createDataFrame(l)

In [366]:
results.printSchema()

root
 |-- source_name: string (nullable = true)
 |-- target_name: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- List_Entidad_Norm: string (nullable = true)



In [330]:
l[0] = l[0]+[None]

In [352]:
first = [1,2,3]

In [354]:
(1,2,3)

(1, 2, 3)

In [219]:
l["Id"]

['237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '237',
 '1886',
 '1886',
 '1886',
 '1886',
 '1886',
 '1886',
 '1886',
 '1886',
 '642',
 '642',
 '642',
 '642',
 '642',
 '642',
 '642',
 '642',
 '900',
 '900',
 '900',
 '900',
 '900',
 '900',
 '900',
 '900',
 '656',
 '656',
 '656',
 '656',
 '656',
 '656',
 '656',
 '656',
 '836',
 '836',
 '836',
 '836',
 '836',
 '836',
 '836',
 '836',
 '860',
 '860',
 '860',
 '860',
 '860',
 '860',
 '860',
 '860',
 '880',
 '880',
 '880',
 '880',
 '880',
 '880',
 '880',
 '880',
 '832',
 '832',
 '832',
 '832',
 '832',
 '832',
 '832',
 '832',
 '891',
 '891',
 '891',
 '891',
 '891',
 '891',
 '891',
 '891',
 '1866',
 '1866',
 '1866',
 '1866',
 '1866',
 '1866',
 '1866',
 '1866',
 '4573',
 '4573',
 '4573',
 '4573',
 '4573',
 '4573',
 '4573',
 '4573',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',
 '861',


In [142]:
tup = tuple("")*len(lo)

In [351]:
Nombre_Input_entidades.toPandas()

,Id,CIF,Entidad_Norm,Provincia_Entidad,ID_ENTIDAD,NIF_COD,ACRONIMO,NOMBRE_ENTIDAD,Nombre_Entidad_Mostrar,TIPO_ENTIDAD_N1_1,...,C_DNM_UD_ORGANICA_EDP_PRINCIPAL,C_ID_ESTADO,D_VIG_ALTA_OFICIAL,NIF_CIF,C_ID_AMB_PROVINCIA,C_DESC_PROV,CONTACTOS,List_Entidad_Norm,List_Provincia_Entidad,List_CIF
0,2668,A36924140,marine instruments sa,36,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['marine instruments sa'],[36],['A36924140']
1,4782,B10398931,controla energia sl,10,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['controla energia sl'],[10],['B10398931']
2,4589,A58567918,optral s a,8,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['optral s a'],[8],['A58567918']
3,3515,A39074877,apia xxi s a,39,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['apia xxi s a'],[39],['A39074877']
4,2891,A41204223,gabinete de iniciativas europeas s a,41,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['gabinete de iniciativas europeas s a'],[41],['A41204223']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8205,7823,Q2868018I,real academia nacional de farmacia,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['real academia nacional de farmacia'],[],['Q2868018I']
8206,7050,B67082586,bettair cities sl,8,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['bettair cities sl'],[8],['B67082586']
8207,3726,A26260596,manufacturas vental s a,26,None,None,None,None,None,None,...,None,None,None,None,None,None,None,['manufacturas vental s a'],[26],['A26260596']
8208,1994,G76515550,asociacion astrofisica para la promocion de la...,28,2045,G76515550,ASPID,ASOCIACION ASTROFISICA PARA LA PROMOCION DE LA...,Asociacion Astrofisica para la Promoción de la...,"Entidad de Gestión, Promoción y Fomento de la I+D",...,None,None,None,None,None,None,None,['asociacion astrofisica para la promocion de ...,[28],['G76515550']


In [141]:
tuple("")+tuple(2)

TypeError: 'int' object is not iterable

In [128]:
k = ['Id', 'CIF']

In [144]:
tuple(k)

('Id', 'CIF')

In [136]:
Nombre_Input_entidades.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [134]:
l[['Id', 'CIF']

TypeError: unhashable type: 'list'

In [131]:
l

{0: {'source_name': 'agencia estatal consejo superior de investigaciones cientificas csic',
  'matching_names_by_elasticsearch': [(465,
    'consejo de la juventud de extremadura'),
   (441, 'agencia de desenvolupament del ripolles'),
   (450, 'centro de computacion cientifica')],
  'Id': '237',
  'CIF': 'Q2818002D'},
 1: {'source_name': 'agencia estatal consejo superior de investigaciones cientificas csic',
  'matching_names_by_elasticsearch': [(465,
    'consejo de la juventud de extremadura'),
   (441, 'agencia de desenvolupament del ripolles'),
   (450, 'centro de computacion cientifica')],
  'Id': '237',
  'CIF': 'Q2818002D'},
 2: {'source_name': 'fundacion parc tauli',
  'matching_names_by_elasticsearch': [(309,
    'instituto de investigacion e innovacion parc tauli'),
   (281, 'fundacion interhospitalaria para la investigacion cardiovascular'),
   (211, 'fundacion para la investigacion puerta del mar')],
  'Id': '1886',
  'CIF': 'G60331238'},
 3: {'source_name': 'fundacion prod

In [125]:
l.keys()

dict_keys(['source_name', 'matching_names_by_elasticsearch', 'Id', 'CIF'])

In [ ]:
l.head(50)

In [67]:
l = {2:"d"}
l.update({1:"s"})


In [68]:
l

{2: 'd', 1: 's'}

In [64]:
Nombre_Input_investigadores.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Entidad: string (nullable = true)
 |-- Origen_Solicitud: string (nullable = true)
 |-- Identificadores_Origen: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- CIF_validacion: string (nullable = true)
 |-- Centro: string (nullable = true)
 |-- Centro_Norm: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Pais_Entidad: string (nullable = true)
 |-- Provincia_Centro: string (nullable = true)
 |-- PIC: string (nullable = true)
 |-- Tipo_Persona: string (nullable = true)
 |-- Entidad_Match: string (nullable = true)
 |-- Provincia_Match: string (nullable = true)
 |-- Match: string (nullable = true)



In [73]:
k = [""] * 2
k[1]="pedrp"
k

['', 'pedrp']

In [5]:
def calcular_Distance_ratcliff_obershelp_ElasticSearch_Ubicacion(df1,
                                                                 source_names,
                                                                 df2,
                                                                 target_names,
                                                                 size=50):
    

    print("entra funcion comparacion")
    df_1 = df1.select(source_names).toPandas()
    df_2 = df2.select(target_names).toPandas()
        

    results = get_matching_entities_by_elasticSearch(df_1,
                                                     source_names,
                                                     df_2,
                                                     target_names,
                                                     index_name='index_name'
                                                    )
    


    schema = StructType([
        StructField('source_names', StringType(), True),
        StructField('target_names', StringType(), True)
    ])

    results = spark.createDataFrame(results, schema)
    
    stopwords_es = nltk.corpus.stopwords.words('spanish')
    
    
    results = results.withColumn("aux", F.split("source_names", "\\s+"))
    remover = StopWordsRemover(stopWords=stopwords_es, inputCol="aux", outputCol="source_names_stopwords")
    results = remover.transform(results).withColumn("source_names_stopwords", F.array_join("source_names_stopwords", " "))

    results = results.withColumn("aux", F.split("target_names", "\\s+"))
    remover = StopWordsRemover(stopWords=stopwords_es, inputCol="aux", outputCol="target_names_stopwords")
    results = remover.transform(results).withColumn("target_names_stopwords", F.array_join("target_names_stopwords", " "))

        
    return results.withColumn('final_score', udf_Distance_ratcliff_obershelp(F.col('source_names_stopwords'), F.col('target_names_stopwords'))).drop('aux').withColumn('final_score', F.when(F.col('source_names_stopwords') != (F.col('target_names_stopwords')), F.col('final_score')).otherwise(F.lit(1))).drop_duplicates()
